### Part 1: Gradient Descent + Example


<div style="text-align: right; direction: rtl;">

### الگوریتم گرادیان کاهشی یک الگوریتم بهینه‌سازی‌ است که برای بهینه‌سازی توابع مشتق‌پذیر استفاده می‌شود
### این الگوریتم معمولا در هر مرحله در جهت معکوس گرادیان تابع هدف حرکت می‌کند تا به نقطه‌ای برسد که گرادیان تابع هدف در آن نقطه صفر شود
### این الگوریتم برای بهینه‌سازی توابع مشتق‌پذیر استفاده می‌شود و می‌تواند بهینه‌سازی توابع غیرخطی را انجام دهد
### در یادگیری‌ماشین این الگوریتم اصولا برای بهینه‌سازی توابع هزینه (که همان اختلاف بین خروجی مدل و خروجی واقعی است) استفاده می‌شود
### فرمول گرادیان کاهشی به صورت زیر است
### $w = w - \alpha \nabla Q(w)$
### که در این فرمول $w$ وزن‌های مدل، $\alpha$ نرخ یادگیری و $\nabla Q(w)$ گرادیان تابع هزینه است
### در این قسمت یک مثال ساده از الگوریتم گرادیان کاهشی ارائه شده است
### فرض کنید یک مدل خطی به‌صورت زیر داریم و می‌خواهیم تتا را به‌روزرسانی کنیم به‌گونه‌ای که خط مدل بهترین تطابق را با داده‌ها داشته باشد
### به‌عبارتی دیگر می‌خواهیم تتا را به‌گونه‌ای به‌روزرسانی کنیم که مقدار تابع هزینه کمینه شود
### $y = \theta x $
### فرض می‌کنیم حدس اولیه ما برای تتا برابر ۳ بوده و نقاط داده‌ای ما به‌صورت زیر است 
### $X = [1, 2, 3, 4]$
### $Y = [2, 4, 6, 8]$
### در این مثال ما می‌خواهیم تتا را به‌روزرسانی کنیم به‌گونه‌ای که خطی که از این تتا به‌دست می‌آید بهترین تطابق را با داده‌ها داشته باشد
### برای این‌کار ابتدا تابع هزینه را تعریف می‌کنیم
### $Q = \frac{1}{2n} \sum_{i=1}^{n} (y_i - \theta x_i)^2$
### سپس گرادیان تابع هزینه را محاسبه می‌کنیم
### $\nabla Q = \frac{-1}{n} \sum_{i=1}^{n} (y_i - \theta x_i) x_i$
### در نهایت تتا را به‌روزرسانی می‌کنیم
### $\theta = \theta - \alpha \nabla Q$
### در این مثال مقدار $\alpha$ را برابر 0.01 در نظر می‌گیریم و یکبار این کار را تکرار می‌کنیم
### میدانیم در عمل تعداد تکرار ها بیش‌تر است 
### در این مثال مقدار گرادیان برابر
### $\nabla Q = \frac{-1}{4} \sum_{i=1}^{4} (y_i - \theta x_i) x_i = \frac{-1}{4} \sum_{i=1}^{4} (y_i - 3 x_i) x_i$
### $= \frac{-1}{4} \sum_{i=1}^{4} (y_i x_i - 3 x_i^2) = \frac{-1}{4} \sum_{i=1}^{4} (2 x_i^2 - 3 x_i^2)$
### $= \frac{-1}{4} \sum_{i=1}^{4} - x_i^2 = \frac{1}{4} \sum_{i=1}^{4} x_i^2$
### $= \frac{1}{4} (1 + 4 + 9 + 16) = \frac{1}{4} \times 30 = 7.5$ 
### پس مقدار جدید تتا برابر است با
### $\theta = 3 - 0.01 \times 7.5 = 3 - 0.075 = 2.925$
### در این‌جا مقدار جدید تتا برابر 2.925 است
### یعنی به مقدار بهینه نزدیک‌تر شدیم
### با تکرار این کار مقدار تتا به مقدار بهینه 2 نزدیک‌تر می‌شود

</div>


In [185]:
import pandas as pd 
import numpy as np
import tqdm
import matplotlib.pyplot as plt

#### Reading Data

In [186]:
df = pd.read_csv('../data/1.csv')

##  Polynomial Regression

<div style="text-align: right; direction: rtl; line-height:3;">


####  مدل رگرسیون چندجمله‌ای
در این بخش می‌خواهیم یک مدل رگرسیون چندجمله‌ای ارائه دهیم که بتواند داده‌های غیرخطی را نیز تخمین بزند
برای این‌کار ابتدا یک مدل خطی ارائه می‌دهیم که به‌صورت زیر است

$y = w_0 + w_1 x_1 + w_2 x_2 + ... + w_n x_n$

سپس این مدل را به‌گونه‌ای تغییر می‌دهیم که بتواند داده‌های غیرخطی را نیز تخمین بزند
برای این‌کار ابتدا تمامی توان‌های ممکن برای هر یک از ویژگی‌ها را محاسبه می‌کنیم
سپس تمامی ترکیب‌های ممکن از این توان‌ها را محاسبه می‌کنیم می‌توانیم این کار را به‌صورت بازگشتی انجام دهیم و تمامی ترکیب‌های ممکن از توان‌ها را به‌دست آوریم می‌دانیم اگر تعداد ویژگی‌ها $n$ باشد و مجموع توان‌ها $m$ باشد تعداد ترکیب‌های ممکن یکتا از توان‌ها برابر است با (مسئله میله‌ها و ستاره‌ها)
 

$\binom{n+m-1}{m - 1}$

سپس ویژگی‌های جدیدی که از ترکیب‌های مختلف توان‌ها به‌دست می‌آید را به‌عنوان ویژگی‌های جدید به مدل خطی اضافه می‌کنیم
در نهایت مدل خطی جدیدی که از این‌جا به‌دست می‌آید مدل رگرسیون چندجمله‌ای است

به‌طور مثال اگر تعداد ویژگی‌ها ۲ باشد و می‌خواهیم توان‌ها تا ۳ را در نظر بگیریم تعداد ترکیب‌های ممکن برابر است با (یک واحد به تعداد ویژگی‌ها اضافه می‌کنیم برای بایاس)

$\binom{3+3-1}{3 - 1} = \binom{5}{2} = 10$

و ترکیب‌های ممکن از توان‌ها به‌صورت زیر است

$[0, 0], [0, 1], [0, 2], [0, 3], [1, 0], [1, 1], [1, 2], [2, 0], [2, 1], [3, 0]$

و ویژگی‌های جدیدی که به‌دست می‌آید به‌صورت زیر است

$[1, x_1, x_2, x_1^2, x_1 x_2, x_2^2, x_1^3, x_1^2 x_2, x_1 x_2^2, x_2^3]$


#### مدل رگرسیون خطی
در این مدل از الگوریتم گرادیان کاهشی برای بهینه‌سازی استفاده می‌کنیم و می‌توانیم مدل رگرسیون خطی را به‌صورت زیر ارائه دهیم

$y = w_0 + w_1 x_1 + w_2 x_2 + ... + w_n x_n$

این مدل را می‌توان به‌صورت ماتریسی زیر نوشت

$\hat{y} = X w$

که در این‌جا $X$ ماتریس ویژگی‌ها و $w$ وزن‌های مدل است

خطای مدل را می‌توان به‌صورت زیر نوشت

$ e = X w - y $

در اینجا $e$ خطای مدل و $y$ خروجی واقعی است

تابع هزینه را به‌صورت ماتریسی می‌توان به‌صورت زیر نوشت

$Q = \frac{1}{2n} e^T e$

که در این‌جا $n$ تعداد نمونه‌ها است

گرادیان تابع هزینه را می‌توان به‌صورت زیر نوشت

$\nabla Q = \frac{1}{n} X^T e$

بدین‌ترتیب وزن‌هارا به‌صورت زیر به‌روزرسانی می‌کنیم 

$w = w - \alpha \nabla Q$

که در این‌جا $\alpha$ نرخ یادگیری است
از رگولاریزیشن $l_1$ برای جلوگیری از بیش‌برازش استفاده می‌کنیم که به‌صورت زیر است

$w = w - \alpha \nabla Q + \lambda \text{sign}(w)$

که در این‌جا $\lambda$ ضریب رگولاریزیشن است

در نهایت مدل را به‌صورت زیر به‌روزرسانی می‌کنیم

$w = w - \alpha \nabla Q + \lambda \text{sign}(w)$

با تکرار این کار مدل بهینه می‌شود
</div>

In [238]:
class MyLinearRegression:
    def __init__(self):
        self.w = None
        
    def fit(self, X: np.ndarray, y: np.array, epochs=100, learning_rate=0.01, lamb=0.01, gd = True):
        rows, cols = X.shape
        if not gd:
            self.w = np.linalg.inv(X.T @ X) @ X.T @ y
        else:
            self.w = np.zeros(cols)
            for _ in tqdm.tqdm(range(epochs)):
                predicts = X @ self.w
                errors = (predicts - y)
                reg = lamb * np.sign(self.w)
                reg[0] = 0
                gradient = X.T @ errors / rows + reg
                update_term = learning_rate  *  gradient
                self.w -= update_term
                
            
        
    def predict(self, X: np.ndarray):
        return X @ self.w

    def score(self, X: np.ndarray, y: np.array):
        y_pred = self.predict(X)
        return  1 - ((y - y_pred)**2).sum() / ((y - y.mean())**2).sum()


class PolynomialRegression: 
    def __init__(self, degree=2, epochs = 100):
        self.degree = degree
        self.epochs = epochs
        self.powers = None
        self.lr = MyLinearRegression()
    
    def fit(self, X: np.ndarray, y: np.array, epochs=100, learning_rate=0.01, gd = True):
        X = self.add_bias(X)
        self.powers = self.get_power_combinations(X.shape[1], self.degree)
        X = self.get_pr_features(X, self.powers)
        self.lr.fit(X, y, epochs=epochs, learning_rate=learning_rate, gd=gd)
        
    def score(self, X: np.ndarray, y: np.array):
        X = self.add_bias(X)
        X = self.get_pr_features(X, self.powers)
        return self.lr.score(X, y)
    
    
    def predict(self, X: np.ndarray):
        X = self.add_bias(X)
        X = self.get_pr_features(X, self.powers)
        return X @ self.lr.w
        
    def add_bias(self, X: np.ndarray): 
        return np.column_stack((np.ones(X.shape[0]) , X))
        
    
    def get_pr_features(self, X: np.ndarray, powers: np.ndarray = None):
        if powers is None:
            powers = self.get_power_combinations(X.shape[1], self.degree)
        return np.prod(np.power(X[:, np.newaxis], powers), axis=-1)


    def get_power_combinations(self, n: int, m: int): 
        if n == 1:
            return [[m]]
        if m == 0:
            return [[0 for _ in range(n)]]
        sols = []
        for i in range(m+1):
            for sol in self.get_power_combinations(n-1, m - i):
                sols.append(
                    [i] + sol
                )
        return sols

<div style="text-align: right; direction: rtl; line-height:3;">

#### توابع زیر را برای جداکردن داده‌ها به دو بخش آموزش و تست ارائه داده‌ایم


</div> 

In [239]:

def train_test_split_manual(df, test_size=0.2, random_state=None, shuffle=True):
    if shuffle:
        df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    if isinstance(test_size, float):
        test_size = int(len(df) * test_size)

    train_df = df.iloc[:-test_size].reset_index(drop=True)
    test_df = df.iloc[-test_size:].reset_index(drop=True)

    return train_df, test_df


In [240]:
NUMERICAL_FEATURES = [
    'Rooms', 'Distance', 'Bedroom2', 'Postcode', 'Bathroom', 'Car',
    'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude', 'Longtitude', 'Propertycount',
]
CATEGORICAL_FEATURES = [
    'Suburb', 'Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname', 'Date'
]
PREDICT = 'Price'

train, test = train_test_split_manual(df, test_size=.2, random_state=40, shuffle=True)

<div style="text-align: right; direction: rtl; line-height:3;">

#### توابع زیر را برای نرمال‌سازی و استاندارد‌سازی داده‌ها ارائه داده‌ایم

#### نرمال‌سازی داده‌ها به‌صورت زیر است

$X_{\text{norm}} = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}$


#### استاندارد‌سازی داده‌ها به‌صورت زیر است

$X_{\text{std}} = \frac{X - \mu}{\sigma}$


</div>

In [241]:
def normalize_df(matrix: np.ndarray):
    return (matrix - matrix.min(axis=0)) / (matrix.max(axis=0) - matrix.min(axis=0))
def standardize_df(matrix: np.ndarray):
    return (matrix - matrix.mean(axis=0)) / matrix.std(axis=0)

In [265]:
from sklearn.impute import SimpleImputer

In [266]:
pr1 = PolynomialRegression(degree=1)
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

train_numeric = imp.fit_transform(train[NUMERICAL_FEATURES])

train_numeric = normalize_df(train_numeric)

y = train[PREDICT].values
pr1.fit(train_numeric, y, epochs=100, learning_rate=.01, gd=True)


test_numeric = imp.fit_transform(test[NUMERICAL_FEATURES])
test_numeric = normalize_df(test_numeric)

y_test = test[PREDICT].values
pr1.score(test_numeric, y_test)

100%|██████████| 100/100 [00:00<00:00, 31536.12it/s]


np.float64(0.02286829088943343)